# FORCE ESTIMATION PREPROCESSING

## Specify Paths and FLAGS

In [ ]:
## TODO
# - unit convert to do but raw embedded into SI units (Si Machine) - DONE
# - jacobian preprocessing using interpolate jacobian (Si Machine) - DONE
# - split embeded data into force sensor ground truth (Si Machine ) - DONE
# - model prediction
# - procesing rosbag data through simons repo dvrk_force_estimation repo (Si Machine )
# - - rosbag reprocess (Si Machine)
# - - preprocess script(interpolations) (Si Machine)


# NOTES
#   make sure you put the json converted files in the jhu-davinci folders  

# file struture
# ---------------
# dataset_name 
# |__joints


## EMBEDDED PATHS
dataset_path = "/home/ndrakes1/force_estimation_data_captures/capture_10-30-25/in_contact_tall_fixture/" # SET
dataset_name = "contact"

fpga_v3_data_collection_path = "/home/ndrakes1/fpgav3-data-collection" # SET THIS ONCE
preproces_path = " "  # should be this directory SET THIS ONCE

true_preprocess_path = "~/force_estimation/preprocess/"


unit_convert_script_py = fpga_v3_data_collection_path + "/unit_convert/unit_convert.py"
si_unit_xml_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM3-698534.xml" # SET THIS ONCE 
si_unit_json_path = "/home/ndrakes1/catkin_ws/src/dvrk/dvrk_config_jhu/jhu-daVinci-Si/sawRobotIO1394-PSM1-292409.xml.json"

zynq_data_csv = "/home/ndrakes1/force_estimation_data_captures/capture_10-30-25/in_contact_tall_fixture/contact.csv" 

unit_convert_path = dataset_path + "/" + dataset_name + "_unitConvert.csv"



ORIGINAL_FREQ = 10000

INTERPOLATE = False


FILTER = True
FILTER_FREQ = 100
FILTER_VELOCITY = True

DOWNSAMPLE = True
DOWNSAMPLE_FREQ = 200
DOWNSAMPLE_MOVING_AVERAGE = True

##################################################################################

# EMBEDDED

In [2]:
!. ~/catkin_ws/devel/setup.bash
from downsample import downsample_dataframe
import pandas as pd 
import os

## Unit Convert

In [3]:
!.venv/bin/python3 {unit_convert_script_py} -c {si_unit_json_path} -f {zynq_data_csv}

Data written to /home/ndrakes1/force_estimation_data_captures/capture_10-30-25/in_contact_tall_fixture/contact_unitConvert.csv


## Extract Force Sensor Data

In [4]:
!mkdir {dataset_path}/sensor
!.venv/bin/python3 sensor_processing.py {zynq_data_csv} {dataset_path}/sensor/sensor.csv

df = pd.read_csv( os.path.join(dataset_path,"sensor","sensor.csv") , header=None)

if DOWNSAMPLE == True:
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    df.to_csv(os.path.join(dataset_path,"sensor","sensor.csv"), index=False, header=False)



Saved SENSOR DATA CSV to /home/ndrakes1/force_estimation_data_captures/capture_10-30-25/in_contact_tall_fixture//sensor/sensor.csv


## Extract Joint Data

In [5]:
!mkdir {dataset_path}joints
!.venv/bin/python3 preprocessing.py {unit_convert_path} {dataset_path}joints/interpolated_all_joints.csv

Traceback (most recent call last):
  File "preprocessing.py", line 25, in <module>
    preprocess_csv(args.fileA, args.output_path)
  File "preprocessing.py", line 6, in preprocess_csv
    df = pd.read_csv(input_csv_path)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 1407, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/home/ndrakes1/force-estimation-fpga-data-preprocess/.venv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 1661, in _make_engine
    self.handles = get_handle(


## Filter Joint Data

### Preprocessed Training File Paths

In [6]:
import os
import pandas as pd

data_file = os.path.join(dataset_path,"joints" ,"interpolated_all_joints.csv")
df = pd.read_csv(data_file, header=None)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ndrakes1/force_estimation_data_captures/capture_10-30-25/in_contact_tall_fixture/joints/interpolated_all_joints.csv'

### Interpolate

In [ ]:
from interpolate_timestamps import interpolate_dataframe_to_sample_rate

if (INTERPOLATE == True):
    
    df = interpolate_dataframe_to_sample_rate(df, ORIGINAL_FREQ)


[0.00000000e+00 4.77575000e-03 9.77616000e-03 ... 4.97847724e+01
 4.97897724e+01 4.97947724e+01]


### FIR Filtering

In [ ]:
import importlib
import filter  # regular import first
importlib.reload(filter)


if (FILTER == True):
    # Design filter
    fir_coeffs = filter.design_fir_filter(filter_type='kaiser', fs=ORIGINAL_FREQ, fC=FILTER_FREQ, order=30)

    # Apply to data
    df = filter.apply_filter_to_torque_feedback_df(df, fir_coeffs, filter_velocity=FILTER_VELOCITY)

### Downsample


In [ ]:


if (DOWNSAMPLE == True):
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)


### SAVE TO CSV

In [ ]:
df.to_csv(data_file, index=False, header=False)


## Extract Jacobian

In [ ]:
!mkdir {dataset_path}jacobian
!.venv/bin/python3 interpolate_jacobian2.py {unit_convert_path} {dataset_path}jacobian/interpolated_all_jacobian.csv dvpsm.rob


FIRST JACOBIAN VALUE
[[ 1.38777878e-17  6.93889390e-18  3.20208314e-02  1.56125113e-17
  -7.16690767e-02 -5.98082934e-02]
 [ 3.46944695e-18 -2.08166817e-17  7.32836073e-01  1.73472348e-18
   3.90551270e-02 -5.16853500e-02]
 [ 0.00000000e+00 -3.46944695e-17 -6.79651349e-01  0.00000000e+00
   3.87347167e-02 -5.85476660e-02]
 [ 5.55111512e-17 -9.98891998e-01  0.00000000e+00  3.20208314e-02
  -6.08006429e-01  7.78903243e-01]
 [-1.00000000e+00 -1.11022302e-16  0.00000000e+00  7.32836073e-01
  -5.25429223e-01 -5.39941300e-01]
 [-2.77555756e-17 -4.70614122e-02  0.00000000e+00 -6.79651349e-01
  -5.95190990e-01 -3.19019013e-01]]
Flattened Jacobians (column-major) written to /home/ndrakes1/force_estimation_data_captures/capture_10-13-25/x_axis_test/jacobian/interpolated_all_jacobian.csv


In [ ]:
df = pd.read_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), header=None)

if DOWNSAMPLE == True:
    df = downsample_dataframe(df, ORIGINAL_FREQ, DOWNSAMPLE_FREQ, DOWNSAMPLE_MOVING_AVERAGE)
    df.to_csv(os.path.join(dataset_path,"jacobian", "interpolated_all_jacobian.csv"), index=False, header=False)

# NOW DO INFERENCE ON THE JOINT DATA ON THE DVRK-12-MACHINE